To run this Fenic demo, click **Runtime** > **Run all**.

<div class="align-center">
<a href="https://github.com/typedef-ai/fenic"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/typedef-fenic-logo-github-yellow.png?raw=true" height="50"></a>
<a href="https://discord.gg/GdqF3J7huR"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/join-the-discord.png?raw=true" height="50"></a>
<a href="https://docs.fenic.ai/latest/"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/documentation.png?raw=true" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/typedef-ai/fenic).

</div>


In [3]:
!pip uninstall -y sklearn-compat ibis-framework imbalanced-learn google-genai
!pip install polars==1.30.0
!pip install huggingface_hub
# === GOOGLE GEMINI ===
!pip install "fenic[google]"
# === ANTHROPIC CLAUDE ===
#!pip install fenic[anthropic]
# === OPENAI (Default) ===
#!pip install "fenic[google]"


Found existing installation: google-genai 1.51.0
Uninstalling google-genai-1.51.0:
  Successfully uninstalled google-genai-1.51.0

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Using cached google_genai-1.51.0-py3-none-any.whl.metadata (46 kB)
Using cached google_genai-1.51.0-py3-none-any.whl (260 kB)

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os 
import getpass

# 🔌 MULTI-PROVIDER SETUP - Choose your preferred LLM provider
# Uncomment provider sections you are using in your semantic config

# === OPENAI (Default) ===
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# === GOOGLE GEMINI ===
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google API Key:")

# === ANTHROPIC CLAUDE ===
# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key:")


# 📄 PDF Processing & Analysis

**Hook:** *"Transform PDFs into structured, queryable data in seconds"*

Research papers, whitepapers, technical documents - PDFs contain valuable information but are notoriously difficult to work with. Traditional PDF processing requires complex parsing, layout analysis, and manual extraction. Watch AI-powered PDF processing convert unstructured documents into structured, searchable data.

**What you'll see in this 2-minute demo:**
- 📚 **PDF to Markdown** - Intelligent conversion preserving structure and formatting
- 🧠 **Content Categorization** - Automatic classification of document sections
- 📊 **Structured Extraction** - Products, training methods, key topics identified
- ⚡ **Batch Processing** - Multiple PDFs processed and analyzed efficiently

Perfect for research analysis, document management, and content discovery.


In [5]:
import fenic as fc
from pydantic import BaseModel, Field
from typing import List
import huggingface_hub as hf
import shutil

# ⚡ Configure for PDF processing with multiple models
session = fc.Session.get_or_create(fc.SessionConfig(
    app_name="pdf_processing_demo",
    semantic=fc.SemanticConfig(
        language_models={
            "parse_model": fc.GoogleDeveloperLanguageModel(
                model_name="gemini-2.5-flash-lite",
                rpm=500,
                tpm=1_000_000,
            ),
            "cheap_model": fc.OpenAILanguageModel(
                model_name="gpt-5-nano",
                rpm=500,
                tpm=200_000,
            ),
        },
        default_language_model="cheap_model"
    )
))

print("✅ PDF processing session configured with Gemini for parsing and GPT-5-nano for analysis")


✅ PDF processing session configured with Gemini for parsing and GPT-5-nano for analysis


## 📄 Step 1: Download Sample PDFs

Let's grab some real whitepapers to process - these are complex technical documents perfect for demonstrating AI-powered PDF analysis.


In [6]:
# 📚 Download sample whitepapers from Hugging Face
data_dir = "sample_pdfs/"
os.makedirs(data_dir, exist_ok=True)

repo_id = "typedef-ai/pdf_data" 
files = hf.list_repo_files(repo_id=repo_id, repo_type="dataset")

print(f"📥 Downloading whitepapers from {repo_id}...")
for file in files:
    if file.startswith("whitepapers/"):
        hf.hf_hub_download(repo_id=repo_id, repo_type="dataset", filename=file, local_dir=data_dir)
        print(f"  ✅ Downloaded: {file}")

print(f"📁 PDFs saved to: {data_dir}")


/Users/davidyoungworth/Projects/fenic2/fenic/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📥 Downloading whitepapers from typedef-ai/pdf_data...
  ✅ Downloaded: whitepapers/PA_whitepaper_10-most-critical-security-risks-in-serverless-architectures-guide.pdf
  ✅ Downloaded: whitepapers/PA_whitepaper_ai-governance.pdf
  ✅ Downloaded: whitepapers/PA_whitepaper_securing-the-soc-with-nec-xon.pdf
  ✅ Downloaded: whitepapers/dedup/.DS_Store
  ✅ Downloaded: whitepapers/dedup/PA_whitepaper_10-most-critical-security-risks-in-serverless-architectures-guide.pdf
  ✅ Downloaded: whitepapers/dedup/PA_whitepaper_ai-governance_copy_0.pdf
  ✅ Downloaded: whitepapers/dedup/PA_whitepaper_ai-governance_copy_1.pdf
  ✅ Downloaded: whitepapers/dedup/PA_whitepaper_ai-governance_copy_2.pdf
  ✅ Downloaded: whitepapers/dedup/PA_whitepaper_securing-the-soc-with-nec-xon.pdf
📁 PDFs saved to: sample_pdfs/


## 🧠 Step 2: AI-Powered PDF to Markdown Conversion

Now the magic happens - watch AI convert complex PDFs into clean, structured markdown while preserving all the important formatting and hierarchy.

First we can filter which documents we parse based on the PDF metadata.  In this case, we're only interested in longer, unencrypted documents.


In [7]:
pdf_filtered_df = session.read.pdf_metadata(f"{data_dir}/**/*.pdf").filter(
    (fc.col("page_count") > 3) & (~fc.col("is_encrypted"))
)

print(f"📊 Found {pdf_filtered_df.count()} valid PDFs to process")
pdf_filtered_df.select("title", "page_count", "file_path").show()


📊 Found 3 valid PDFs to process
┌──────────────────────────────────────────┬────────────┬──────────────────────────────────────────┐
│ title                                    ┆ page_count ┆ file_path                                │
╞══════════════════════════════════════════╪════════════╪══════════════════════════════════════════╡
│ Securing the Modern SOC with NEC XON and ┆ 6          ┆ sample_pdfs/whitepapers/PA_whitepaper_se │
│ Palo Alto Networks                       ┆            ┆ curing-the-soc-with-nec-xon.pdf          │
│ AI Governance for AI-Powered             ┆ 14         ┆ sample_pdfs/whitepapers/PA_whitepaper_ai │
│ Applications                             ┆            ┆ -governance.pdf                          │
│ 10 Most Critical Security Risks in       ┆ 17         ┆ sample_pdfs/whitepapers/PA_whitepaper_10 │
│ Serverless Architectures guide           ┆            ┆ -most-critical-security-risks-in-serverl │
│                                          ┆            ┆ e

In [ ]:
# 🚀 Convert PDFs to Markdown using AI
print("🤖 Converting PDFs to markdown using Gemini...")
pdf_to_md_content = pdf_filtered_df.with_column(
    "markdown_content", 
    fc.semantic.parse_pdf(fc.col("file_path"), model_alias="parse_model")
).cache()

print("✅ PDF to Markdown conversion complete!")
print(f"📄 Processed {pdf_to_md_content.count()} documents")


🤖 Converting PDFs to markdown using Gemini...
✅ PDF to Markdown conversion complete!


Submitting requests for batch: c1d9a84d-d7dd-435f-91ec-06e77843a350 (model: gemini-2.5-flash-lite): 100%|██████████| 13/13 [00:00<00:00, 75.04req/s, estimated_input_tokens=9546, estimated_output_tokens=20845]
Awaiting responses for batch c1d9a84d-d7dd-435f-91ec-06e77843a350 (model: gemini-2.5-flash-lite):   0%|          | 0/13 [00:00<?, ?res/s]

## 📊 Step 3: Extract Document Structure

Fenic's powerful markdown processing can extract any structure from the converted content. Let's break down documents into sections and generate table of contents.


In [ ]:
# 📋 Extract document structure and table of contents
pdf_md_content = pdf_to_md_content.select(
    fc.when(
        fc.col("title").is_not_null(), 
        fc.col("title")
    ).otherwise(
        fc.text.split_part(fc.col("file_path"), "/", -1)
    ).alias("name"),
    "markdown_content",
    # Generate table of contents
    fc.markdown.generate_toc(fc.col("markdown_content")).alias("toc")
)

# Extract sections up to level 3 headers
pdf_sections_1 = pdf_md_content.with_column("sections", fc.markdown.extract_header_chunks(fc.col("markdown_content"), header_level=1))
pdf_sections_2 = pdf_md_content.with_column("sections", fc.markdown.extract_header_chunks(fc.col("markdown_content"), header_level=2))
pdf_sections_3 = pdf_md_content.with_column("sections", fc.markdown.extract_header_chunks(fc.col("markdown_content"), header_level=3))
pdf_sections_df = pdf_sections_2.union(pdf_sections_3)


print("📊 Document structure extracted:")
pdf_sections_df.select("name", "sections", "toc").show()
pdf_sections_df.to_polars()["toc"][2]



📊 Document structure extracted:
┌────────────────────────────────┬────────────────────────────────┬────────────────────────────────┐
│ name                           ┆ sections                       ┆ toc                            │
╞════════════════════════════════╪════════════════════════════════╪════════════════════════════════╡
│ AI Governance for AI-Powered   ┆ [{"Current AI Landscape and    ┆ # Establishing a Governance    │
│ Applications                   ┆ Its Security                   ┆ Framework for AI-Powered       │
│                                ┆ Implications",2,"The AI        ┆ Applications                   │
│                                ┆ landscape has changed          ┆ ## Current AI Landscape and    │
│                                ┆ dramatically in recent years,  ┆ Its Security Implications      │
│                                ┆ with significant advancements  ┆ ### "Traditional" AI and       │
│                                ┆ in both general and     

'# Top 10 Critical Security Risks in Serverless Architectures\n# Serverless Security Overview\n# Top 10 Risks in Serverless Architecture Security\n# The 10 Most Critical Security Risks in Serverless Architectures\n## SAS-1: Function Event Data Injection\n### Putting Theory into Practice: An Injection Flaw Scenario\n### When Trusted Data Sources Become an Attack Vector\n# Comparison\n## Mitigation\n## SAS-2: Broken Authentication\n### Comparison\n## Mitigation\n## SAS-3: Insecure Serverless Deployment Configuration\n# Comparison\n## Mitigation\n## SAS-4: Overprivileged Function Permissions and Roles\n## Mitigation\n## SAS-5: Inadequate Function Monitoring and Logging\n## Mitigation\n## SAS-6: Insecure Third-Party Dependencies\n### Comparison\n### Mitigation\n## SAS-7: Insecure Application Secrets Storage\n### Comparison\n### Mitigation\n## SAS-8: Denial of Service and Financial Resource Exhaustion\n### Comparison\n### Mitigation\n## SAS-9: Serverless Function Execution Flow Manipulation

## 🧠 Step 4: AI-Powered Content Analysis

Now let's use AI to analyze the content and extract structured insights - what products are mentioned, what sections discuss model training, and more.


In [ ]:
# 🎯 Define content categorization schema
class PDFContentCategorization(BaseModel):
    """AI-powered PDF content categorization."""
    summary: str = Field(description="Brief one sentence summary of the PDF given its table of contents")
    sections_about_model_training: List[str] = Field(description="List of headings that are specifically about model training")
    sections_about_soc_compliance: List[str] = Field(description="List of headings that are related to SOC compliance")
    sections_about_data_governance: List[str] = Field(description="List of headings that are related to data governance")
    products_mentioned: List[str] = Field(description="All product names mentioned in the PDF table of contents")

print("🎯 Content categorization schema defined")


🎯 Content categorization schema defined


In [ ]:
# 🤖 AI-powered content analysis using table of contents
pdf_filtered_details = pdf_sections_df.with_column(
    "content_categorization", 
    fc.semantic.extract(fc.col("toc").cast(fc.StringType), PDFContentCategorization, model_alias="cheap_model")
).cache()

print("✅ AI content analysis complete!")

#pdf_filtered_details.to_polars()

✅ AI content analysis complete!


## 📊 Step 5.1: Summarize the PDFs content

Let's see what insights AI extracted from our PDFs - summaries, products mentioned, and training-related sections.


In [ ]:
# 📊 Display whitepaper summaries and insights
print("="*70)
print("📄 WHITEPAPER ANALYSIS RESULTS")
print("="*70)


for row in pdf_filtered_details.to_pylist():
    print(f"\n📚 Whitepaper: {row['name']}")
    print(f"📝 Summary: {row['content_categorization']['summary']}")
    print(f"🏷️  Products mentioned: {row['content_categorization']['products_mentioned']}")
    print(f"🧠 Training sections: {row['content_categorization']['sections_about_model_training']}")
    print(f"🔒 SOC compliance sections: {row['content_categorization']['sections_about_soc_compliance']}")
    print(f"🔒 Data Governance sections: {row['content_categorization']['sections_about_data_governance']}")
    print("-" * 50)



📄 WHITEPAPER ANALYSIS RESULTS


Submitting requests for batch: 36fcb70c-6b1a-4388-898c-9e51696c04c2 (model: gpt-5-nano): 100%|██████████| 6/6 [00:00<00:00, 643.36req/s, estimated_input_tokens=1788, estimated_output_tokens=9216]
Awaiting responses for batch 36fcb70c-6b1a-4388-898c-9e51696c04c2 (model: gpt-5-nano): 100%|██████████| 6/6 [00:03<00:00,  1.72res/s]


📚 Whitepaper: AI Governance for AI-Powered Applications
📝 Summary: A governance-focused document outlining AI risk, security implications, and a framework for managing AI-powered applications, with emphasis on model governance, data governance, use cases, compliance, and security tooling.
🏷️  Products mentioned: ['Cortex Cloud AI Security Posture Management', 'Palo Alto Networks']
🧠 Training sections: ['Model Training', 'Typical Use Cases', 'Security Implications', 'Fine-Tuning and Retrieval-Augmented Generation (RAG)', 'Typical Use Cases', 'Security Implications']
🔒 SOC compliance sections: ['Compliance', 'Oversight of Relevant Compliance Frameworks', 'Ongoing Consideration of Current and Future Compliance Risk']
🔒 Data Governance sections: ['Data', 'Discovery and Classification Data Used for AI Model Training and Deployment', 'Policies to Prevent Data Poisoning', 'Policies Regarding the Use of Sensitive Data for Training, Inference, and Fine-Tuning']
--------------------------------

## 🔍 Step 5.2: Deep Dive into Training Sections

Imagine you're a data security engineer and you're only interested in Data Governence and SOC compliance.  We can cheaply filter content based on the markdown headings and only examine relevant parts of the whitepapers!


In [ ]:
# 🔍 Filter sections specifically about model training
soc_compliance_sections_df = pdf_filtered_details.explode("sections").filter(
    fc.col("sections").is_not_null() &
    fc.col("content_categorization").is_not_null() &
    (fc.array_contains(fc.col("content_categorization").sections_about_soc_compliance, fc.col("sections").heading) |
    fc.array_contains(fc.col("content_categorization").sections_about_data_governance, fc.col("sections").heading))

)

    
print("="*70)
print("🧠 SOC COMPLIANCE AND DATA GOVERNANCE DEEP DIVE")
print("="*70)
print(f"📊 Found {soc_compliance_sections_df.count()} sections about SOC compliance or Data Governance:")
print()

# Display training sections by document
for row in soc_compliance_sections_df.group_by("name").agg(fc.collect_list("sections").alias("sections")).to_pylist():
    print(f"📚 Document: {row['name']}")
    for section in row['sections']:
        print(f"  📖 Section: {section['heading']}")
        print(f"  📝 Content preview: {section['content'][:200]}...")
        print()
    print("-" * 50)


🧠 SOC COMPLIANCE AND DATA GOVERNANCE DEEP DIVE
📊 Found 4 sections about SOC compliance or Data Governance:

📚 Document: AI Governance for AI-Powered Applications
  📖 Section: Data
  📝 Content preview: Data is the lifeblood of AI models. It is therefore critical to have clear visibility into what data is being used across the AI life cycle. This includes data used for training models, for inference ...

  📖 Section: Oversight of Relevant Compliance Frameworks
  📝 Content preview: AI governance does not exist in a vacuum but must be integrated with an organization's overall compliance management framework. This means aligning AI policies and controls with relevant laws, regulat...

  📖 Section: Ongoing Consideration of Current and Future Compliance Risk
  📝 Content preview: The regulatory landscape around AI is undergoing major changes. New laws and standards are being proposed and enacted, often with significant implications regarding how organizations develop and deplo...

  📖 Section:

## 🎉 What Just Happened?

**You just witnessed the future of document processing:**

1. **📄 PDF → Markdown**: AI converted complex PDFs into clean, structured markdown while preserving formatting and hierarchy
2. **🧠 Content Analysis**: AI analyzed document structure and extracted key insights like products mentioned and training sections  
3. **📊 Structured Data**: Transformed unstructured PDFs into queryable, structured data
4. **🔍 Smart Filtering**: Automatically identified and extracted only relevant sections

**This is semantic AI in action** - understanding document content, not just extracting text. Perfect for research analysis, document management, and content discovery.

**Try this with your own PDFs** - just change the huggingface directory or use your own s3 bucket and watch fenic work it's magic!


In [ ]:
# 🧹 Cleanup
print("🧹 Cleaning up downloaded files...")
shutil.rmtree(data_dir)
session.stop()
print("✅ Cleanup complete!")


🧹 Cleaning up downloaded files...

Session Usage Summary:
  App Name: pdf_processing_demo
  Session ID: 684cd7d9-07ff-4ad3-963d-e2e8718990be
  Total queries executed: 8
  Total execution time: 13460.27ms
  Total rows processed: 37
  Total language model cost: $0.008766
  Total language model requests: 16
  Total language model tokens: 10,641 input tokens, 2,992 cached input tokens, 19,555 output tokens
  Total embedding model cost: $0.00
  Total cost: $0.008766
✅ Cleanup complete!
